In [1]:
"""Booking subscriptions and redemptions

Demonstration of how to model subscriptions and redemptions in LUSID

Attributes
----------
transaction configuration
cocoon - seed_data
holdings
"""

'Booking subscriptions and redemptions\n\nDemonstration of how to model subscriptions and redemptions in LUSID\n\nAttributes\n----------\ntransaction configuration\ncocoon - seed_data\nholdings\n'

## Booking subscriptions and redemptions

Here we show how you can use LUSID's transaction type model to book subscriptions and redemptions. From a high level, we decompose these cash transactions into their underlying economic movements:

* <b>Subscriptions</b>: These increase the cash holding of a given currency (e.g. CCY_GBP or CCY_JPY) in a fund.
* <b>Redemptions</b>: These decrease the cash holding of a given currency in a fund.




### Setup LUSID

In [2]:
# Import LUSID
import lusid.models as models
from lusidjam import RefreshingToken
import fbnsdkutilities.utilities as utils

# Import Libraries
import pprint
import pytz
import pandas as pd
import numpy as np
import json
import requests
import os
import warnings
import lusid
import lusidtools.cocoon.cocoon as cocoon
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import format_transactions_response
from datetime import datetime, timedelta, time

warnings.filterwarnings("ignore")

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = utils.ApiClientFactory(
    lusid,
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version:  0.6.11251.0


### Load transaction file and external mappings

In [3]:
# Define a unqique scope
scope = "use-case-nb-subs-reds"
portfolio_code = "GLOBAL-EQUITY"

# Load a mapping file for loading data
with open(r"config/seed_data.json") as mappings_file:
    seed_data_mapping = json.load(mappings_file)

# Load a file to format holding response
with open(r"config/format_holdings_response.json") as mappings_file:
    format_holdings_response = json.load(mappings_file)

# Load transaction file
transactions_file = r"data/global_equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

seed_transactions = transactions_df[transactions_df["txn_trade_date"] == "01/04/2020"]

In [4]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    seed_transactions,
    "DataFrame",
    mappings=seed_data_mapping,
)

print(f"Portfolio {portfolio_code} has been created with transactions.")

Portfolio GLOBAL-EQUITY has been created with transactions.


### Check holdings for 10 April 2020

In [5]:
# Define the transactions portfolio API
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

In [6]:
# Call the get transactions method


def get_holdings(date):

    get_holdings_response = transaction_portfolios_api.get_holdings(
        scope=scope,
        code=portfolio_code,
        effective_at=date,
        property_keys=["Instrument/default/Name"],
    )

    holdings_df = lusid_response_to_data_frame(
        get_holdings_response,
        rename_properties=True,
        column_name_mapping=format_holdings_response,
    )

    return holdings_df


get_holdings("2020-04-10")

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,AmountCost,CurrencyCost,CostPortfolioCcyAmount,CostPortfolioCcyCurrency,currency,holding_type_name
0,default,LUID_KR3A1NMI,{},Aviva,GLOBAL-EQUITY,use-case-nb-subs-reds,P,1000000.0,1000000.0,1000000.0,GBP,1000000.0,GBP,GBP,Position
1,default,LUID_SIMWQCNR,{},Barclays,GLOBAL-EQUITY,use-case-nb-subs-reds,P,1000000.0,1000000.0,1000000.0,GBP,1000000.0,GBP,GBP,Position
2,default,LUID_80DILFAS,{},BP,GLOBAL-EQUITY,use-case-nb-subs-reds,P,3000000.0,3000000.0,3000000.0,GBP,3000000.0,GBP,GBP,Position
3,default,CCY_GBP,{},GBP,GLOBAL-EQUITY,use-case-nb-subs-reds,B,10000000.0,10000000.0,10000000.0,GBP,10000000.0,GBP,GBP,Balance


### We have some subscriptions and redemptions in May

Let's look in our file:

* 2 million GBP out of the fund on 01 May
* 5 million USD into the fund on 01 May 

For the purposes of this notebook, we use the system default transaction types of `FundsIn` (a subscription) and `FundsOut` (a redemption). However you are free to configure your own transaction type aliases. See the linked [tutorial](https://support.finbourne.com/how-do-i-create-holdings#config-txn-types) for further details.

In [7]:
subs_reds = transactions_df[transactions_df["txn_trade_date"] > "01/04/2020"]
subs_reds["portfolio_code"] = portfolio_code
subs_reds

,portfolio_code,portfolio_name,portfolio_base_currency,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,cash_transactions
5,GLOBAL-EQUITY,A generic global Equity portfolio,GBP,cash,GBP,GBP Cash,cash_002,FundsOut,01/05/2020,03/05/2020,2000000,1,2000000,GBP,GBP
6,GLOBAL-EQUITY,A generic global Equity portfolio,USD,cash,USD,USD Cash,cash_003,FundsIn,01/05/2020,03/05/2020,5000000,1,5000000,USD,USD


### Load these subs and reds into LUSID

In [8]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Currency": "cash_transactions",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

In [9]:
result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=subs_reds,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
print(f"number of successful portfolios requests: {len(succ)}")
print(f"number of failed portfolios requests    : {len(failed)}")

number of successful portfolios requests: 1
number of failed portfolios requests    : 0


### Check the holdings again on 10 May 2020

Now we have:

* Some Equity holdings
* A balance of 8 million GBP
* A balance of 5 million USD

In [10]:
get_holdings("2020-05-10")

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,AmountCost,CurrencyCost,CostPortfolioCcyAmount,CostPortfolioCcyCurrency,currency,holding_type_name
0,default,LUID_KR3A1NMI,{},Aviva,GLOBAL-EQUITY,use-case-nb-subs-reds,P,1000000.0,1000000.0,1000000.0,GBP,1000000.0,GBP,GBP,Position
1,default,LUID_SIMWQCNR,{},Barclays,GLOBAL-EQUITY,use-case-nb-subs-reds,P,1000000.0,1000000.0,1000000.0,GBP,1000000.0,GBP,GBP,Position
2,default,LUID_80DILFAS,{},BP,GLOBAL-EQUITY,use-case-nb-subs-reds,P,3000000.0,3000000.0,3000000.0,GBP,3000000.0,GBP,GBP,Position
3,default,CCY_GBP,{},GBP,GLOBAL-EQUITY,use-case-nb-subs-reds,B,8000000.0,8000000.0,8000000.0,GBP,8000000.0,GBP,GBP,Balance
4,default,CCY_USD,{},USD,GLOBAL-EQUITY,use-case-nb-subs-reds,B,5000000.0,5000000.0,5000000.0,USD,0.0,GBP,USD,Balance
